In [ ]:
import opencc

# 创建一个转换器
converter = opencc.OpenCC('t2s')

# 繁体中文字符串
traditional_text = "綺殿千尋起，離宮百雉餘。"
# 转换为简体中文
simplified_text = converter.convert(traditional_text)

print(simplified_text)

In [ ]:
import json
import os

#https://www.heywhale.com/home/global?search=%E5%85%A8%E5%94%90%E8%AF%97%26%E5%85%A8%E5%AE%8B%E8%AF%97%E6%95%B0%E6%8D%AE%E9%9B%86
#这个数据集包含5.5万首唐诗和26万首宋诗，以及相关的作者信息，存储为JSON文件。每个JSON文件包含1000篇诗，数据以繁体字存储，
#但您可能需要将其转换为简体字以适应transformers库的要求。

# 假设您的JSON文件存放在'path_to_json_files'目录下
#json_files_dir = 'C:\\Users\\drhu0\\data\\poetry\\poet.tang.7000.json'
json_files_dir = 'C:\\Users\\drhu0\\data\\poetry\\'
for filename in os.listdir(json_files_dir):
    if filename.endswith('.json'):
        file_path = os.path.join(json_files_dir, filename)
        # 读取JSON文件
        print(file_path)
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)

        # 对数据集中的文本进行转换
        for poem in data:
            if 'paragraphs' in poem:
                poem['paragraphs'] = [converter.convert(text) for text in poem['paragraphs']]
                poem['title'] = converter.convert(poem['title'])
                if 'author' in poem:
                    poem['author'] = converter.convert(poem['author'])

        
        # 将转换后的数据写回文件
        with open(file_path, 'w', encoding='utf-8') as file:
            json.dump(data, file, ensure_ascii=False, indent=4)

In [15]:
import json
import os
import opencc

# 创建一个转换器
converter = opencc.OpenCC('t2s')

#https://www.heywhale.com/home/global?search=%E5%85%A8%E5%94%90%E8%AF%97%26%E5%85%A8%E5%AE%8B%E8%AF%97%E6%95%B0%E6%8D%AE%E9%9B%86
#这个数据集包含5.5万首唐诗和26万首宋诗，以及相关的作者信息，存储为JSON文件。每个JSON文件包含1000篇诗，数据以繁体字存储，
#但您可能需要将其转换为简体字以适应transformers库的要求。

# 假设您的JSON文件存放在'path_to_json_files'目录下
#json_files_dir = 'C:\\Users\\drhu0\\data\\poetry\\poet.tang.7000.json'

json_files_dir = 'C:\\Users\\drhu0\\data\\poetry\\'
# 保存为 poems.txt 文件
with open('poems.txt', 'a', encoding='utf-8') as f:
    for filename in os.listdir(json_files_dir):
        if filename.endswith('.json'):
            file_path = os.path.join(json_files_dir, filename)
            # 读取JSON文件
            print(file_path)
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
    
            # 对数据集中的文本进行转换
            for poem in data:
                if 'paragraphs' in poem:
                    poem['paragraphs'] = [converter.convert(text) for text in poem['paragraphs']]
                    content = '\n'.join(poem['paragraphs'])
                    poem['title'] = converter.convert(poem['title'])
                    title = poem['title']
                    if 'author' in poem:
                        poem['author'] = converter.convert(poem['author'])
                        author = poem['author']

                    f.write(f"{title} {author}\n{content}\n\n")
            
            
            # 将转换后的数据写回文件
            with open(file_path, 'w', encoding='utf-8') as file:
                json.dump(data, file, ensure_ascii=False, indent=4)

C:\Users\drhu0\data\poetry\authors.song.json
C:\Users\drhu0\data\poetry\authors.tang.json
C:\Users\drhu0\data\poetry\poet.song.0.json
C:\Users\drhu0\data\poetry\poet.song.1000.json
C:\Users\drhu0\data\poetry\poet.song.10000.json
C:\Users\drhu0\data\poetry\poet.song.100000.json
C:\Users\drhu0\data\poetry\poet.song.101000.json
C:\Users\drhu0\data\poetry\poet.song.102000.json
C:\Users\drhu0\data\poetry\poet.song.103000.json
C:\Users\drhu0\data\poetry\poet.song.104000.json
C:\Users\drhu0\data\poetry\poet.song.105000.json
C:\Users\drhu0\data\poetry\poet.song.106000.json
C:\Users\drhu0\data\poetry\poet.song.107000.json
C:\Users\drhu0\data\poetry\poet.song.108000.json
C:\Users\drhu0\data\poetry\poet.song.109000.json
C:\Users\drhu0\data\poetry\poet.song.11000.json
C:\Users\drhu0\data\poetry\poet.song.110000.json
C:\Users\drhu0\data\poetry\poet.song.111000.json
C:\Users\drhu0\data\poetry\poet.song.112000.json
C:\Users\drhu0\data\poetry\poet.song.113000.json
C:\Users\drhu0\data\poetry\poet.song.

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import pandas as pd

# Load and prepare the dataset
def load_poem_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    poems = []
    poem = ""
    for line in lines:
        line = line.strip()
        if line:
            poem += line + " "
        else:
            poems.append(poem.strip())
            poem = ""
    if poem:
        poems.append(poem.strip())

    return pd.DataFrame(poems, columns=["text"])

# Convert DataFrame to Dataset
file_path = 'C:\\Users\\drhu0\\tang_poems.txt'
df = load_poem_data(file_path)
print(df.head())  # Display the first few rows of the DataFrame

# The DataFrame should look something like this:
#                                                  text
# 0  静夜思 李白 床前明月光，疑是地上霜。 举头望明月，低头思故乡。
# 1  春晓 孟浩然 春眠不觉晓，处处闻啼鸟。 夜来风雨声，花落知多少。
# 2  登鹳雀楼 王之涣 白日依山尽，黄河入海流。 欲穷千里目，更上一层楼。

dataset = Dataset.from_pandas(df)

# Check the dataset structure
#print("Dataset columns:", dataset.column_names)

# Load tokenizer and model
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Add a padding token if not already present
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = AutoModelForCausalLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

# Tokenize the dataset
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs

# Apply the tokenization function to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Set the format for PyTorch
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    eval_steps=500,
    save_total_limit=2,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
)

# Train the model
trainer.train()

# Generate a poem
def generate_poem(prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors='pt')
    outputs = model.generate(inputs.input_ids, max_length=max_length, num_return_sequences=1, pad_token_id=tokenizer.pad_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
prompt = "[BOS]床"
generated_poem = generate_poem(prompt)
print(generated_poem)


                                                text
0                   静夜思 李白 床前明月光，疑是地上霜。 举头望明月，低头思故乡。
1                   春晓 孟浩然 春眠不觉晓，处处闻啼鸟。 夜来风雨声，花落知多少。
2                 登鹳雀楼 王之涣 白日依山尽，黄河入海流。 欲穷千里目，更上一层楼。
3              横吹曲辞 洛阳陌 李白 白玉谁家郎，回车渡天津。 看花东上陌，惊动洛阳人。
4  横吹曲辞 长安道 崔颢 长安甲第高入云，谁家居住霍将军。 日晚朝回拥宾从，路傍拜揖何纷纷。 ...


Map:   0%|          | 0/2008 [00:00<?, ? examples/s]

Step,Training Loss
10,70.332000
20,60.912400
30,53.453000
40,37.168500
50,17.554600
60,7.279000
70,4.581100
80,4.090400
90,3.730800
100,3.570500


KeyboardInterrupt: 

In [ ]:
#获取唐诗数据集
#你可以从以下来源获取大量的唐诗数据集：

#网络爬虫：可以编写爬虫从网上的唐诗数据库中爬取数据，如中国古诗词网。https://www.gushiwen.cn/
#现成数据集：可以在一些数据平台上找到现成的唐诗数据集，如Kaggle或GitHub。
#例如，在GitHub上可以找到一些唐诗数据集：

#chinese-poetry: 这个项目包含了大量的中国古诗，包括唐诗、宋词等。https://github.com/chinese-poetry/chinese-poetry
#以下是一个简单的脚本，展示如何从 chinese-poetry 项目中提取唐诗并保存为 tang_poems.txt 文件。

"""
静夜思 李白
床前明月光，疑是地上霜。
举头望明月，低头思故乡。

春晓 孟浩然
春眠不觉晓，处处闻啼鸟。
夜来风雨声，花落知多少。

登鹳雀楼 王之涣
白日依山尽，黄河入海流。
欲穷千里目，更上一层楼。
"""

import json

# 下载 chinese-poetry 项目中的唐诗数据集
#!git clone https://github.com/chinese-poetry/chinese-poetry.git
#"C:\Users\drhu0\chinese-poetry"

# 读取唐诗数据
#poems_file_path = 'C:\\Users\\drhu0\\data\\poetry\\poet.tang.7000.json'
poems_file_path = 'C:\\Users\\drhu0\\data\\poetry\\poet.tang.1000.json'
with open(poems_file_path, 'r', encoding='utf-8') as f:
    poems_data = json.load(f)

# 保存为 tang_poems.txt 文件
with open('tang_poems_1000.txt', 'w', encoding='utf-8') as f:
    for poem in poems_data:
        title = poem['title']
        author = poem['author']
        content = '\n'.join(poem['paragraphs'])
        f.write(f"{title} {author}\n{content}\n\n")

print("唐诗数据集已保存为 tang_poems.txt 文件")


In [10]:
# Example usage
prompt = "[BOS]春眠"
generated_poem = generate_poem(prompt)
print(generated_poem)

[BOS]春眠辞 刘长卿卿 刘长卿 刘长卿 刘长卿，�


In [12]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset, Dataset

# Load and prepare the dataset
def load_poem_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    poems = [line.strip() for line in lines if line.strip()]
    return Dataset.from_dict({'text': poems})

# Create dataset
file_path = 'C:\\Users\\drhu0\\poems.txt'
dataset = load_poem_data(file_path)

# Load pre-trained GPT-2 model and tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add special tokens to the tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]', 'bos_token': '[BOS]', 'eos_token': '[EOS]'})

# Resize the model embeddings to accommodate the new tokens
model.resize_token_embeddings(len(tokenizer))

# Define the tokenization function
def encode(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

# Encode the dataset
dataset = dataset.map(encode, batched=True)

# Ensure that the labels are set for language modeling
def set_labels(examples):
    examples['labels'] = examples['input_ids'].copy()
    return examples

dataset = dataset.map(set_labels, batched=True)
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Split the dataset into train and test sets
train_test_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10000,
    save_steps=500,
    eval_steps=500,
    save_total_limit=2,
)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Start training
trainer.train()

# Function to generate poems
def generate_poem(prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors='pt')
    outputs = model.generate(inputs.input_ids, max_length=max_length, num_return_sequences=1, pad_token_id=tokenizer.pad_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example generation
prompt = "[BOS]"
generated_poem = generate_poem(prompt)
print(generated_poem)


Map:   0%|          | 0/12227 [00:00<?, ? examples/s]

Map:   0%|          | 0/12227 [00:00<?, ? examples/s]

Step,Training Loss
10,15.915400
20,14.285700
30,13.098300
40,10.556700
50,7.117000
60,3.287300
70,2.822800
80,1.410700
90,1.230000
100,1.108000


KeyboardInterrupt: 

In [7]:
from transformers import BertTokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset, Dataset

# Load the dataset
file_path = 'C:\\Users\\drhu0\\poems.txt'
dataset = load_dataset('text', data_files=file_path)

# Each example in the dataset should have a single 'text' key
dataset = dataset['train'].map(lambda x: {'text': x['text']}, batched=True)

# Load the tokenizer and model
model_name = "uer/gpt2-chinese-cluecorpussmall"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add special tokens if necessary
tokenizer.add_special_tokens({'pad_token': '[PAD]', 'bos_token': '[BOS]', 'eos_token': '[EOS]'})
model.resize_token_embeddings(len(tokenizer))

# Tokenize the dataset
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Prepare the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10000,
    save_steps=500,
    eval_steps=500,
    save_total_limit=2,
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
)

# Start training
trainer.train()

# Generate a poem
def generate_poem(prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors='pt')
    outputs = model.generate(inputs.input_ids, max_length=max_length, num_return_sequences=1, pad_token_id=tokenizer.pad_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
prompt = "[BOS]床前明月光"
generated_poem = generate_poem(prompt)
print(generated_poem)


Map:   0%|          | 0/1990602 [00:00<?, ? examples/s]

Step,Training Loss
10,13.448600
20,12.811300
30,11.586400
40,9.566500


KeyboardInterrupt: 

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
import torch.optim as optim

# 唐诗数据集类
class TangPoetryDataset(Dataset):
    def __init__(self, file_path):
        self.lines = self.load_data(file_path)
    
    def load_data(self, file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()
        return [line.strip() for line in lines if line.strip()]

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, idx):
        line = self.lines[idx]
        inputs = self.encode_line(line)
        return inputs

    def encode_line(self, line):
        inputs = tokenizer(line, return_tensors='pt', truncation=True, padding='max_length', max_length=128)
        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'token_type_ids': inputs['token_type_ids'].squeeze(),
            'labels': torch.tensor(0)  # Dummy label
        }

# Load the tokenizer and model
model_name = 'bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 准备数据集和数据加载器
file_path = 'C:\\Users\\drhu0\\poems.txt'
dataset = TangPoetryDataset(file_path)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 3  # Set your number of epochs
total_steps = len(dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Train the model
model.train()
for epoch in range(num_epochs):
    for batch in dataloader:
        inputs = {k: v.to(device) for k, v in batch.items()}
        
        optimizer.zero_grad()
        outputs = model(input_ids=inputs['input_ids'],
                        attention_mask=inputs['attention_mask'],
                        token_type_ids=inputs['token_type_ids'])
        
        labels = inputs['labels']  # Use the dummy labels for now
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

# 唐诗生成函数
def generate_poem(model, tokenizer, prompt, max_length=128):
    inputs = tokenizer(prompt, return_tensors='pt', padding='max_length', max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_token_id = logits.argmax(dim=-1).item()
        
    generated_text = tokenizer.decode([predicted_token_id], skip_special_tokens=True)
    return generated_text

# 使用模型生成唐诗
prompt = "春眠不觉晓"
generated_poetry = generate_poem(model, tokenizer, prompt)
print(f"生成的唐诗是: {generated_poetry}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\drhu0\miniconda3\Lib\site-packages\transformers\optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Loss: 0.7547716498374939
Epoch 1, Loss: 0.5879641771316528
Epoch 1, Loss: 0.45749133825302124
Epoch 1, Loss: 0.44901230931282043
Epoch 1, Loss: 0.3923766613006592
Epoch 1, Loss: 0.3453485667705536
Epoch 1, Loss: 0.2975447177886963
Epoch 1, Loss: 0.26130154728889465
Epoch 1, Loss: 0.24867664277553558
Epoch 1, Loss: 0.2203451544046402
Epoch 1, Loss: 0.212896466255188
Epoch 1, Loss: 0.19444090127944946
Epoch 1, Loss: 0.16372805833816528
Epoch 1, Loss: 0.15276332199573517
Epoch 1, Loss: 0.14179085195064545
Epoch 1, Loss: 0.12659871578216553
Epoch 1, Loss: 0.12024648487567902
Epoch 1, Loss: 0.10622072964906693
Epoch 1, Loss: 0.10353100299835205
Epoch 1, Loss: 0.08363205194473267
Epoch 1, Loss: 0.08071460574865341
Epoch 1, Loss: 0.07418403774499893
Epoch 1, Loss: 0.06778427213430405
Epoch 1, Loss: 0.061483461409807205
Epoch 1, Loss: 0.06031893566250801
Epoch 1, Loss: 0.05322068929672241
Epoch 1, Loss: 0.05321302264928818
Epoch 1, Loss: 0.04571026563644409
Epoch 1, Loss: 0.0451198220

KeyboardInterrupt: 

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForMaskedLM, get_linear_schedule_with_warmup
from torch.optim import AdamW

# 唐诗数据集类
class TangPoetryDataset(Dataset):
    def __init__(self, file_path):
        self.lines = self.load_data(file_path)
    
    def load_data(self, file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()
        return [line.strip() for line in lines if line.strip()]

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, idx):
        line = self.lines[idx]
        inputs = self.encode_line(line)
        return inputs

    def encode_line(self, line):
        inputs = tokenizer(line, return_tensors='pt', truncation=True, padding='max_length', max_length=128)
        inputs['labels'] = inputs['input_ids'].clone()
        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'token_type_ids': inputs['token_type_ids'].squeeze(),
            'labels': inputs['labels'].squeeze()
        }

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertForMaskedLM.from_pretrained('bert-base-chinese')

# Prepare the dataset and dataloader
file_path = 'C:\\Users\\drhu0\\poems.txt'
dataset = TangPoetryDataset(file_path)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 3  # Set your number of epochs
total_steps = len(dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Train the model
model.train()
for epoch in range(num_epochs):
    for batch in dataloader:
        inputs = {k: v.to(device) for k, v in batch.items()}
        
        outputs = model(input_ids=inputs['input_ids'],
                        attention_mask=inputs['attention_mask'],
                        token_type_ids=inputs['token_type_ids'],
                        labels=inputs['labels'])
        loss = outputs.loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

# 唐诗生成函数
def generate_poem(model, tokenizer, prompt, max_length=128):
    model.eval()
    inputs = tokenizer(prompt, return_tensors='pt', truncation=True, padding='max_length', max_length=128).to(device)
    with torch.no_grad():
        outputs = model.generate(inputs['input_ids'], max_length=max_length, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# 使用模型生成唐诗
prompt = "春眠不觉晓"
generated_poetry = generate_poem(model, tokenizer, prompt)
print(f"生成的唐诗是: {generated_poetry}")


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1, Loss: 16.88946533203125
Epoch 1, Loss: 15.399680137634277
Epoch 1, Loss: 14.003783226013184
Epoch 1, Loss: 12.763955116271973
Epoch 1, Loss: 13.108798027038574
Epoch 1, Loss: 11.630894660949707
Epoch 1, Loss: 11.961350440979004
Epoch 1, Loss: 11.367018699645996
Epoch 1, Loss: 11.147954940795898
Epoch 1, Loss: 10.482924461364746
Epoch 1, Loss: 10.247537612915039
Epoch 1, Loss: 9.710864067077637
Epoch 1, Loss: 9.426915168762207


KeyboardInterrupt: 

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import pandas as pd

# Load and prepare the dataset
def load_poem_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    poems = []
    poem = ""
    for line in lines:
        line = line.strip()
        if line:
            poem += line + " "
        else:
            poems.append(poem.strip())
            poem = ""
    if poem:
        poems.append(poem.strip())

    return pd.DataFrame(poems, columns=["text"])

# Convert DataFrame to Dataset
file_path = 'C:\\Users\\drhu0\\poems.txt'
df = load_poem_data(file_path)
print(df.head())  # Display the first few rows of the DataFrame

dataset = Dataset.from_pandas(df)

# Check the dataset structure
print("Dataset columns:", dataset.column_names)

# 选择一个 GPT-Neo 模型的变种，例如 'EleutherAI/gpt-neo-1.3B'，'EleutherAI/gpt-neo-2.7B' 等
model_name = 'EleutherAI/gpt-neo-1.3B'
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Add a padding token if not already present
tokenizer.add_special_tokens({'pad_token': '[PAD]', 'bos_token': '[BOS]', 'eos_token': '[EOS]'})
model.resize_token_embeddings(len(tokenizer))

# Tokenize the dataset
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs

# Apply the tokenization function to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Set the format for PyTorch
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10000,
    save_steps=500,
    eval_steps=500,
    save_total_limit=2,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
)

# Train the model
trainer.train()

# Generate a poem
def generate_poem(prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors='pt')
    outputs = model.generate(inputs.input_ids, max_length=max_length, num_return_sequences=1, pad_token_id=tokenizer.pad_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
prompt = "[BOS]床前明月光"
generated_poem = generate_poem(prompt)
print(generated_poem)


                                                text
0           日诗 宋太祖 欲出未出光辣达，千山万山如火发。 须臾走向天上来，逐却残星赶却月。
1                             句 宋太祖 未离海底千山黑，才到天中万国明。
2  登戎州江楼闲望 幸夤逊 满目江山四望幽，白云高卷嶂烟收。 日回禽影穿疏木，风递猿声入小楼。 ...
3            雪 幸夤逊 片片飞来静又闲，楼头江上复山前。 飘零尽日不归去，帖破清光万里天。
4  云 幸夤逊 因登巨石知来处，勃勃元生绿藓痕。 静即等闲藏草木，动时顷刻徧乾坤。 横天未必朋元...
Dataset columns: ['text']


Map:   0%|          | 0/311899 [00:00<?, ? examples/s]

Step,Training Loss


KeyboardInterrupt: 